In [1]:
%%time
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry import Registry
import pandas as pd, re, json

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
CPU times: user 1.46 s, sys: 715 ms, total: 2.17 s
Wall time: 3.94 s


In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [3]:
session = get_session('DIAMONDS',"fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [4]:
import requests
exp_id="600b9f90-f431-4590-83c7-f38fe138dedc"#"600b9f90-f431-4590-83c7-f38fe138dedc"
project_id="68bce134-6d7a-4e24-9591-86266438acf9"
executions_url = f"http://monitor-backend-service/monitor/api/v4/executions/search?objectId={exp_id}&projectId={project_id}&jobType=EXPERIMENT&profile=DETAILED"

temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "68bce134-6d7a-4e24-9591-86266438acf9"
}

In [5]:
response = requests.get(executions_url, headers=temp_head)
print(response.json())
res = response.json()

{'totalRecordCount': 1, 'data': [{'jobinstanceId': 110003, 'objectId': '600b9f90-f431-4590-83c7-f38fe138dedc', 'projectId': '68bce134-6d7a-4e24-9591-86266438acf9', 'projectName': None, 'jobObjectName': 'New_Experiment_01', 'jobScheduleId': None, 'jobInstanceInputParameter': '{"input_params":"{\\"currentTime\\":\\"2024-07-10T14:10:56Z\\",\\"objectName\\":\\"New_Experiment_01\\",\\"lastRunDate\\":\\"2024-07-10T13:45:54Z\\",\\"instanceId\\":\\"110003\\",\\"lastSuccessfulRunDate\\":\\"\\",\\"userId\\":\\"aditya1.singh2@fosfor.com\\"}"}', 'jobType': 'EXPERIMENT', 'jobStatus': 'SUCCESSFUL', 'finalStatus': 'SUCCESSFUL', 'jobCreatedby': 'aditya1.singh2@fosfor.com', 'jobCreatedts': '2024-07-10T14:10:56.270+00:00', 'jobUpdatedby': 'aditya1.singh2@fosfor.com', 'jobUpdatedts': '2024-07-10T14:11:41.460+00:00', 'applicationId': '01b59392-0000-72c9-0000-576d074ae99a', 'executionId': None, 'runID': '110003', 'versionNumber': None, 'runEngine': 'SNOWFLAKE', 'attemptId': 110107, 'workflowId': None, 'job

In [8]:
_res_format = {
            "run_id": "",
            "algorithm_used": "",
            "run_status": "",
        }
monitor_resp = []

In [10]:
for data in res.get('data'):
    jobConfig=data.get('jobConfig')
    pattern = r"snowflake\.ml[^']*"
    match = re.search(pattern, jobConfig)
    if match:
        algorithm_used = match.group().rsplit('.', 1)
    _res_format.update({"run_id": {data.get('runID')},
                        "run_status": data.get('jobStatus'),
                        "algorithm_used": algorithm_used[1] if match else None})
    res_data = _res_format
    monitor_resp.append(res_data)

In [11]:
query = "SELECT " \
        "METADATA:metrics:model_metrics as model_metrics," \
        "METADATA:metrics:run_id as run_id, " \
        "METADATA:metrics:status as status, " \
        "MODEL_NAME as sf_model_name " \
        f"FROM INFORMATION_SCHEMA.MODEL_VERSIONS where METADATA:metrics:experiment_id='{exp_id}';"

In [12]:
query

"SELECT METADATA:metrics:model_metrics as model_metrics,METADATA:metrics:run_id as run_id, METADATA:metrics:status as status, MODEL_NAME as sf_model_name FROM INFORMATION_SCHEMA.MODEL_VERSIONS where METADATA:metrics:experiment_id='600b9f90-f431-4590-83c7-f38fe138dedc';"

In [13]:
registry_resp = session.sql(query).to_pandas()
monitor_resp = pd.DataFrame(monitor_resp)
print(monitor_resp)
registry_resp.rename(columns={'MODEL_METRICS': 'model_metrics',
                              'RUN_ID': 'run_id',
                              'STATUS': 'status',
                              'SF_MODEL_NAME': 'experiment_model_name'}, inplace=True)  
print(registry_resp)
final_resp = pd.merge(monitor_resp, registry_resp, on='run_id')
# final_resp = monitor_resp.join(registry_resp, how='left')
a = {'data': final_resp.to_dict(orient='records')}

     run_id algorithm_used  run_status
0  {110003}  XGBClassifier  SUCCESSFUL
                                       model_metrics    run_id     status  \
0  {\n  "accuracy_score": 0.840549,\n  "f1_score"...  "110003"  "Trained"   

                               experiment_model_name  
0  EXPERIMENT_68BCE134_6D7A_4E24_9591_86266438ACF...  


TypeError: unhashable type: 'set'

In [ ]:
print(monitor_resp)
print(registry_resp)
print(a)

In [ ]:
import json, re
res=response.json()
for data in res.get('data'):
    jobConfig=data.get('jobConfig')
    print(jobConfig)
#     data = json.loads(data)
#     print(jobConfig)
    pattern = r'snowflake\.ml[^"]*'
    x = re.search(pattern, jobConfig)
    print(x.group(0))
    print(type(data))
    break

In [ ]:
%%time
monitor_resp = [
    {
        "run_id": "RUN_0001",
        "algorithm_used": "XGBClassifier",
        "status": "Successful"
    },
    {
        "run_id": "RUN_0001_11",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful"
    },
    {
        "run_id": "ef47a0a5",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running"
    },
    {
        "run_id": "26a15f9c",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running"
    }
]

# monitor_resp = []
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a") #,"")0e0fb803-22db-4d88-9f2f-f6f75b6abcf0
query ="SELECT METADATA:metrics as model_metrics, COMMENT as run_id FROM INFORMATION_SCHEMA.MODEL_VERSIONS where MODEL_NAME like '%EXP_001%';"
registry_resp = session.sql(query).to_pandas()
registry_resp.rename(columns={'RUN_ID': 'run_id', 
                              'MODEL_METRICS': 'model_metrics'}, inplace=True)
monitor_resp = pd.DataFrame(monitor_resp)
final_resp = pd.merge(monitor_resp, registry_resp, on='run_id', how='left')
print({'data': final_resp.to_dict(orient='records')})

In [ ]:
monitor_resp=[]
if monitor_resp:
    print("TRUE")
else:
    print("NOT TRUE")

In [ ]:
%%time
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": {}
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": {}
    }
]

def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a")
reg = Registry(session=session)

model_list = session.sql("show models like '%exp6789%'").collect()
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    for run in monitor_resp:
        if run.get('run_id') == model_run_id:
            run.update(metrics)
print(monitor_resp)

In [ ]:
%%time
####Approach 2####
monitor_resp = [
    {
        "run_id": "8042932c-101a-4ddc-a808-00c092797960",
        "algorithm_used": "XGBClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
        "algorithm_used": "DecisionTreeClassifier",
        "status": "Successful",
        "model_metrics": {}
    },
    {
        "run_id": "ef47a0a5-1a74-4a14-b0fd-8dbbd322057f",
        "algorithm_used": "GradientBoostingClassifier",
        "status": "Running",
        "model_metrics": {}
    },
    {
        "run_id": "26a15f9c-29fd-4d55-82fb-3d38cc18e4b8",
        "algorithm_used": "RandomForestClassifier",
        "status": "Running",
        "model_metrics": {}
    }
]

def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a")
reg = Registry(session=session)
resp_df = pd.DataFrame(monitor_resp)
model_list = session.sql("show models like '%exp6789%'").collect()
for model in model_list:
    model_name = model["name"]
    model_run_id = model_name[-36:].replace('_','-').lower()
    model_obj = reg.get_model(model_name)
    metrics = {
        "model_metrics" : model_obj.default.get_metric('model_metrics')
    }
    select_row = resp_df.loc[resp_df['run_id']==model_run_id]
    resp_df.loc[resp_df['run_id']==model_run_id, 'model_metrics'] = str(metrics.get('model_metrics'))   
print(resp_df)

In [ ]:
import requests, json

In [ ]:
monitor_url = "http://monitor-backend-service/monitor/api/v5/executions/search?objectId=f40619bc-16a1-4d6b-8f74-6b95c1d7c76a_bWFpbg==_U25vd3BhcmtSbkQvQnVpbGRfRGF0YXNldC5pcHluYg==&projectId=0e0fb803-22db-4d88-9f2f-f6f75b6abcf0&jobType=NOTEBOOK"
temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "0e0fb803-22db-4d88-9f2f-f6f75b6abcf0"
}

In [ ]:
res = requests.get(monitor_url, headers=temp_head)
res = json.loads(res.text)

In [ ]:
res

In [ ]:
class ResponseFormats:
    LIST_RUNS = {
            "run_id": "",
            "algorithm_used": "",
            "status": "",
            "model_metrics": {}
            }

In [ ]:
_res_format = ResponseFormats.LIST_RUNS

In [ ]:
_res_format['status']

In [ ]:
final_resp = []
for data in res.get('data'):
    _res_format.update({"run_id": data.get('runID'), "status": data.get('jobStatus')})
    res_data = _res_format
    final_resp.append(res_data)

print(final_resp)

In [ ]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex
        
session = get_session("EMPLOYEE_10L","fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [ ]:
reg = Registry(session=session)

In [ ]:
model_name = 'FD0CF79C_8118_43A7_8F0B_059E8F78227A_EXP_001_RUN_0001'

In [ ]:
model_obj = reg.get_model(model_name)
metrics = model_obj.default.show_metrics()


In [ ]:
metrics

In [ ]:
dataset = model_obj.default.get_metric('dataset')

In [ ]:
dataset

In [ ]:
! pip freeze | grep -i snowf